keyword_extraction_using_proportion_ratio.ipynb 파일에서 만들어둔 sparse matrix인 x와 {word:index}인 dictionary type word2index, list of str type인 index2word를 pickle로부터 로딩했습니다. 

In [1]:
import pickle

with open('params', 'rb') as f:
    params = pickle.load(f)
    x = params['x']
    index2word = params['index2word']
    word2index = params['word2index']

In [2]:
import sys
sys.path.append('../')
import soykeyword

Lasso Logistic regression을 이용한 키워드 추출 방법은 proportion ratio 를 이용하는 방법과 그 개념은 비슷합니다. 

Proportion ratio 방법에서 score가 1.0에 가깝다는 의미는 reference documents 보다 target documents에서 등장하는 비율이 높다는 의미이고, 이는 target / reference documents를 구분하는 질좋은 features라는 의미이기도 합니다. 

LassoKeywordExtractor는 Lasso Logistic regression을 이용하여 이런 features를 직접적으로 선택하는 것입니다. 이를 위해 scikit-learn의 Logistic Regression을 이용하였습니다. 이 때 parameters로 입력해야 하는 C는 costs라는 list of number로 입력됩니다. 키워드의 후보들은 모두 min_tf, min_df로 필터링이 됩니다. 

LassoKeywordExtractor는 텍스트 형식의 데이터는 지원하지 않습니다. Sparse matrix 형식에만 지원하며, train 단계에서 index2word를 입력하지 않으면 <키워드 아이디, 빈도수, Logistic regression coefficient> 형식의 namedtuple인 KeywordScore로 출력됩니다. 

minimum_number_of_keywords는 L1 regularization cost를 조금씩 낮춰가면서 선택되는 최소한의 키워드의 개수가 minimum_number_of_keywords 이상이 되도록 하는 최초의 keywords를 return하도록 하는 parameter입니다. 

In [3]:
from soykeyword.lasso import LassoKeywordExtractor

lassobased_extractor = LassoKeywordExtractor(costs=[500, 200, 100, 50, 10, 5, 1, 0.1],
                                             min_tf=20, 
                                             min_df=10)

lassobased_extractor.train(x)
keywords = lassobased_extractor.extract_from_word(5537, minimum_number_of_keywords=30)
keywords[:10]

28 keywords extracted from 0.100 cost
63 keywords extracted from 1.000 cost


[KeywordScore(word=2308, frequency=86, coefficient=3.8101342096180626),
 KeywordScore(word=4701, frequency=40, coefficient=3.2615992235172864),
 KeywordScore(word=4441, frequency=90, coefficient=2.4439388545593541),
 KeywordScore(word=5880, frequency=221, coefficient=1.7592672175198703),
 KeywordScore(word=8215, frequency=105, coefficient=1.4943386150403641),
 KeywordScore(word=4343, frequency=46, coefficient=1.4180686410546366),
 KeywordScore(word=2913, frequency=119, coefficient=1.3447427704258959),
 KeywordScore(word=3527, frequency=195, coefficient=1.1863913809567048),
 KeywordScore(word=3261, frequency=216, coefficient=1.1689093834338116),
 KeywordScore(word=6208, frequency=77, coefficient=1.1161946233047406)]

index2word를 x와 함께 train()에 넣는다면 키워드 추출 결과는 모두 word index가 str로 바뀌어서 입력됩니다. 만약 word index가 index2word의 boundary를 넘어가는 경우라면 Unk101과 같이 Unk%d의 형식으로 출력됩니다. 

In [ ]:
from soykeyword.lasso import LassoKeywordExtractor

lassobased_extractor = LassoKeywordExtractor(costs=[500, 200, 100, 50, 10, 5, 1, 0.1],
                                             min_tf=20, 
                                             min_df=10)

lassobased_extractor.train(x, index2word)

index2word가 train에 입력되면 '아이오아이'가 포함된 문서의 index를 가져올 수도 있습니다. 

In [5]:
documents = lassobased_extractor.get_document_index('아이오아이')

extract_from_docs(documents)는 documents를 positive set으로, documents가 아닌 모든 문서를 negative set으로 선택한 뒤, 이 둘을 구분하는 L1 Logistic regression을 학습합니다. 이 때에는 '아이오아이'라는 단어가 포함된 문서와 포함되지 않는 문서를 구분하는 classifier를 학습하였기 때문에 '아이오아이'가 최상위 키워드로 선택됩니다. 하지만 이는 정답을 알고 문제를 푸는 격입니다. extract_from_docs는 이 경우보다는 본인이 positive set으로 선택하고 싶은 문서 집합이 명확히 있을 때 이용하는 방법입니다.  

In [6]:
keywords = lassobased_extractor.extract_from_docs(documents, minimum_number_of_keywords=30)
keywords

4 keywords extracted from 0.100 cost
4 keywords extracted from 1.000 cost
4 keywords extracted from 5.000 cost
5 keywords extracted from 10.000 cost
6 keywords extracted from 50.000 cost
20 keywords extracted from 100.000 cost
31 keywords extracted from 200.000 cost


[KeywordScore(word='아이오아이', frequency=270, coefficient=17.850189941320671),
 KeywordScore(word='엠카운트다운', frequency=221, coefficient=1.200759338786378),
 KeywordScore(word='뮤직', frequency=195, coefficient=1.081777863860977),
 KeywordScore(word='일산동구', frequency=36, coefficient=0.98636875892070186),
 KeywordScore(word='키미', frequency=297, coefficient=0.70877507721215616),
 KeywordScore(word='챔피언', frequency=105, coefficient=0.51940928356916138),
 KeywordScore(word='강렬', frequency=352, coefficient=0.36972563098092176),
 KeywordScore(word='컴백', frequency=536, coefficient=0.30677481146665397),
 KeywordScore(word='화려', frequency=518, coefficient=0.26764304959838653),
 KeywordScore(word='수출', frequency=735, coefficient=0.23882691530127598),
 KeywordScore(word='걸그룹', frequency=1060, coefficient=0.20972098801573957),
 KeywordScore(word='방영', frequency=208, coefficient=0.19694219657704334),
 KeywordScore(word='프로듀스101', frequency=96, coefficient=0.17074232136595247),
 KeywordScore(word='카메라', fr

extract_from_word(aspect_word)는 aspect_word가 들어간 문서 집합을 positive set으로, 그렇지 않은 문서 집합을 negative set으로 선택하여 이를 구분하는 classifier를 학습하는 것입니다. 이 때, positive set에서 aspect_word를 제외합니다. 이렇게 할 경우, aspect_word 때문에 잘못된 hyperplane이 선택되는 문제를 해결할 수 있습니다. 

2016-10-20에는 '아이오아이'가 마지막 곡인 '너무너무너무'곡으로 활동하던 시절입니다. 그렇기 때문에 '너무너무너무'와 '엠카운트다운'과 같은 단어가 키워드로 선택되었음을 볼 수 있습니다. 

In [7]:
keywords = lassobased_extractor.extract_from_word('아이오아이', minimum_number_of_keywords=30)
keywords

28 keywords extracted from 0.100 cost
63 keywords extracted from 1.000 cost


[KeywordScore(word='너무너무너무', frequency=86, coefficient=3.8159005957233778),
 KeywordScore(word='선의', frequency=40, coefficient=3.2584820410431181),
 KeywordScore(word='산들', frequency=90, coefficient=2.4407245228574896),
 KeywordScore(word='엠카운트다운', frequency=221, coefficient=1.7601587420428146),
 KeywordScore(word='챔피언', frequency=105, coefficient=1.4864913827165669),
 KeywordScore(word='사나', frequency=46, coefficient=1.4183641861333143),
 KeywordScore(word='드림', frequency=119, coefficient=1.3338856375792103),
 KeywordScore(word='뮤직', frequency=195, coefficient=1.1767179765646125),
 KeywordScore(word='먹고', frequency=216, coefficient=1.1632972589808017),
 KeywordScore(word='완전체', frequency=77, coefficient=1.121112062888608),
 KeywordScore(word='일산', frequency=194, coefficient=0.96056172786240313),
 KeywordScore(word='세련', frequency=192, coefficient=0.95771231821225888),
 KeywordScore(word='같이', frequency=918, coefficient=0.85412065320926667),
 KeywordScore(word='컴백', frequency=536, coef

'최순실'의 경우에도 '박근혜-최순실 게이트'사건이 시작되는 단계이기 때문에 해당 뉴스들이 나오고 있었습니다. '최순실'이라는 단어가 들어간 문서를 선택하는 기준은 [게이트, 정유라, 고영태, 이화여대, 미르재단]과 같은 단어가 선택되어 해당 날짜의 뉴스를 잘 요약하고 있음을 볼 수 있습니다. 

In [8]:
keywords = lassobased_extractor.extract_from_word('최순실', minimum_number_of_keywords=30)
keywords

77 keywords extracted from 0.100 cost


[KeywordScore(word='게이트', frequency=303, coefficient=4.2039744027059394),
 KeywordScore(word='정유라', frequency=329, coefficient=2.036813215953313),
 KeywordScore(word='고영태', frequency=65, coefficient=1.7210239759252404),
 KeywordScore(word='편파기소', frequency=54, coefficient=1.2855144476156439),
 KeywordScore(word='최서원', frequency=94, coefficient=1.2799071911280708),
 KeywordScore(word='비선', frequency=288, coefficient=1.0250321401288136),
 KeywordScore(word='씨와', frequency=372, coefficient=0.78303104867933293),
 KeywordScore(word='이화여대', frequency=651, coefficient=0.72298667752697188),
 KeywordScore(word='송민순', frequency=821, coefficient=0.68942381242963791),
 KeywordScore(word='뉴스', frequency=4682, coefficient=0.57871477523586123),
 KeywordScore(word='연설문', frequency=204, coefficient=0.49710754572260307),
 KeywordScore(word='침묵', frequency=223, coefficient=0.42648827071523282),
 KeywordScore(word='박근혜', frequency=1445, coefficient=0.40753898309866521),
 KeywordScore(word='실세', frequency=